# Setup Slapd

Validate `slapd.conf`

In [6]:
! slaptest -u -f slapd.conf

config file testing succeeded


Validate data.ldif

In [10]:
! ldapadd -v -n -f data.ldif

add objectclass:
	top
	organization
	domainRelatedObject
	dcobject
add dc:
	example
add l:
	Anytown, Michigan
add st:
	Michigan
add o:
	Example, Inc.
	EX
	Ex.
add description:
	The Example, Inc. at Anytown
add postaladdress:
	Example, Inc. $ 535 W. William St. $ Anytown, MI 48109 $ US
add telephonenumber:
	+1 313 555 1817
add associateddomain:
	example.com
!adding new entry "dc=example,dc=com"

add objectclass:
	organizationalUnit
	extensibleObject
add ou:
	People
add uidNumber:
	0
add gidNumber:
	0
!adding new entry "ou=People,dc=example,dc=com"

add objectclass:
	organizationalUnit
add ou:
	Groups
!adding new entry "ou=Groups,dc=example,dc=com"

add objectclass:
	organizationalUnit
add ou:
	Alumni Association
!adding new entry "ou=Alumni Association,ou=People,dc=example,dc=com"

add objectclass:
	organizationalUnit
add ou:
	Information Technology Division
add description:
	NOT ASCII (4976 bytes)
	NOT ASCII (2983 bytes)
!adding new entry "ou=Information Technology Division,ou=People,d

Populate database

In [16]:
! slapd -Ta -f slapd.conf -l data.ldif

Closing DB...


Start Slapd.

In [1]:
! slapd -f slapd.conf -h ldap://localhost:9011

Check if Slapd is running.

In [2]:
! lsof -P -i -n | grep slapd

slapd     1656            root    7u  IPv6  14655      0t0  TCP [::1]:9011 (LISTEN)
slapd     1656            root    8u  IPv4  14656      0t0  TCP 127.0.0.1:9011 (LISTEN)


# SlapModRdn

Show original info of entry `cn=James A Jones 1`.

In [3]:
! ldapsearch -x -LLL -b 'cn=James A Jones 1, ou=Alumni Association, ou=People, dc=example, dc=com' -H ldap://localhost:9011/ 

dn: cn=James A Jones 1,ou=Alumni Association,ou=People,dc=example,dc=com
objectClass: OpenLDAPperson
cn: James A Jones 1
cn: James Jones
cn: Jim Jones
sn: Jones
uid: jaj
postalAddress: Alumni Association $ 111 Maple St $ Anytown, MI 48109
seeAlso: cn=All Staff,ou=Groups,dc=example,dc=com
userPassword:: amFq
homePostalAddress: 3882 Beverly Rd. $ Anytown, MI 48105
homePhone: +1 313 555 4772
description: Outstanding
title: Mad Cow Researcher, UM Alumni Association
pager: +1 313 555 3923
mail: jaj@mail.alumni.example.com
facsimileTelephoneNumber: +1 313 555 4332
telephoneNumber: +1 313 555 0895



Change cn of entry: `cn=James A Jones 1` --> `cn=James A Jones X`

In [4]:
! ldapmodrdn -x -D cn=Manager,dc=example,dc=com -H ldap://localhost:9011/ -w secret 'cn=James A Jones 1, ou=Alumni Association, ou=People, dc=example, dc=com' 'cn=James A Jones X'

Check if cn of the entry is changed.

In [6]:
! ldapsearch -P 3 -x -LLL -S '' -b dc=example,dc=com -H ldap://localhost:9011/ 'cn=James A Jones X'

dn: cn=James A Jones X,ou=Alumni Association,ou=People,dc=example,dc=com
objectClass: OpenLDAPperson
cn: James A Jones 1
cn: James Jones
cn: Jim Jones
cn: James A Jones X
sn: Jones
uid: jaj
postalAddress: Alumni Association $ 111 Maple St $ Anytown, MI 48109
seeAlso: cn=All Staff,ou=Groups,dc=example,dc=com
userPassword:: amFq
homePostalAddress: 3882 Beverly Rd. $ Anytown, MI 48105
homePhone: +1 313 555 4772
description: Outstanding
title: Mad Cow Researcher, UM Alumni Association
pager: +1 313 555 3923
mail: jaj@mail.alumni.example.com
facsimileTelephoneNumber: +1 313 555 4332
telephoneNumber: +1 313 555 0895



Revert cn of the entry to original.

In [8]:
! ldapmodrdn -P 3 -x -D cn=Manager,dc=example,dc=com -r -H ldap://localhost:9011/ -w secret 'cn=James A Jones X, ou=Alumni Association, ou=People, dc=example, dc=com' 'cn=James A Jones 1'

Check if the cn is reverted.

In [10]:
! ldapsearch -P 3 -x -LLL -S '' -b dc=example,dc=com -H ldap://localhost:9011/ 'cn=James A Jones 1'

dn: cn=James A Jones 1,ou=Alumni Association,ou=People,dc=example,dc=com
objectClass: OpenLDAPperson
cn: James A Jones 1
cn: James Jones
cn: Jim Jones
sn: Jones
uid: jaj
postalAddress: Alumni Association $ 111 Maple St $ Anytown, MI 48109
seeAlso: cn=All Staff,ou=Groups,dc=example,dc=com
userPassword:: amFq
homePostalAddress: 3882 Beverly Rd. $ Anytown, MI 48105
homePhone: +1 313 555 4772
description: Outstanding
title: Mad Cow Researcher, UM Alumni Association
pager: +1 313 555 3923
mail: jaj@mail.alumni.example.com
facsimileTelephoneNumber: +1 313 555 4332
telephoneNumber: +1 313 555 0895

